## Uplift 
Uplift Model是一个增量模型，目的是预测某种干预对于个体状态或行为的因果效应

<img src="../data/img/uplift.jpeg" style="zoom:50%" />


Uplift models预测增量值，也就是lift的部分：

lift = P(buy|treatment) - P(buy|notreatment)

<img src="../data/img/uplift_model.png" />


### Uplift value

<img src="../data/img/uplift_value.png" />



### Uplift Model
增益模型通常用于用于估算CATE/ITE，基础的uplift model有以下三种：

- Two-Model：建立两个对于outcome的预测模型，一个用实验组数据、一个用对照组数据。
- One-Model：Class Variable Transformation：响应结果为二元变量时可用。
- 对uplift直接建模：对现有ML模型（树、RF、SVM）的改造

#### 双模型 Two-Model Approach
**通常作为baseline模型。**

对 $E[Y_i(1)|X_i]  和  E[Y_i(0)|X_i]$ 分别建模，一个用户通过两个模型进行预测并对结果取差则得到预估的uplift值：
- 优点：简单易用，可以使用常用的回归/分类模型
- 缺点：不是对uplift直接建模，可能会错失一些uplift相关的信号、导致表现较差

<img src="../data/img/uplift_two_model.png" style="zoom:50%"/>

最简单的Uplift建模方法是基于Two Model的差分响应模型，它的形式和前面介绍的Uplift Model的定义非常相似，包含了两个响应模型，其中一个模型G用来估计用户在有干预情况下的响应，另外一个模型G'是用来学习用户在没有干预情况下的响应，之后将两个模型的输出做差，就得到我们想要的uplift。这种建模方法的优点是比较简单容易理解，同时它可以套用我们常见的机器学习模型，如LR，GBDT，NN等，所以该模型的落地成本是比较低的，但是该模型最大的缺点是精度有限，这一方面是因为我们独立的构建了两个模型，这两个模型在打分上面的误差容易产生累积效应，第二是我们建模的目标其实是response而不是uplift，因此对uplift的识别能力比较有限。


### One Model：The Class Transformation Method

适用于 $Y_i^{obj} = \{0,1\}$ 的二分类场景。

目标函数 $Z_i =  Y_i^{obj} W_i + (1 - Y_i^{obj})(1 - W_i)$\
$W_i^{obj} = \{0,1\}， 1是实验组，0是对照组$ \
$Z_i$ 两种情况下等于1（其他时候为0）
- 对象在实验组($W_i = 1$)中且 $Y_i^{obj}$ = 1 => $Z_i = W_i = 1$ 
- 对象在对照组($W_i = 0$)中且 $Y_i^{obj}$ = 0 => $Z_i = 1 - W_i = 1$ 



满足条件$p(X_i)=P(W_i=1|X_i) = 1/2 $ （个体被分到实验组和对照组的概率相同）的情况下，可以证明uplift为\
$$\tau(X_i) = 2P(Z_i=1|X_i) - 1$$

$
\displaystyle
\begin{align}
\tau(X_i) &= P(Y_i=1|X_i, T=1) - P(Y_i=1|X_i, T=0)\\
          &= P(Y_i=1|X_i, T=1) - (1 - P(Y_i=0|X_i, T=0))\\
          &= P(Z_i=1|X_i) - (1 - P(Z_i=1|X_i))\\
          &= 2P(Z_i=1|X_i) - 1\\
\end{align}
$
  

缺点：需要满足两个假设——1. 二分类场景，2. 数据在实验/对照组的分布一致，较为严格\
<img src="../data/img/uplift_one_model.png" style="zoom:50%"/>

进一步地，还有一个基于One Model的差分响应模型，它和上一个模型最大差别点在于，它在模型层面做了打通，同时底层的样本也是共享的，之所以能实现这种模型层面的打通，是因为我们在样本的维度上做了一个扩展，除了user feature之外，还引入了与treatment相关的变量T ( T如果是0，1的取值可以建模single treatment，T也可以扩展为0到N，建模multiple treatment，比如不同红包的面额，或者不同广告的素材 )。

One Model版本和Two Model版本相比最大的优点是**训练样本的共享可以使模型学习的更加充分，同时通过模型的学习也可以有效的避免双模型打分误差累积的问题**，另外一个优点是从模型的层面可以支持multiple treatment的建模，具有比较强的实用性。同时和Two Model版本类似，它的缺点依然是其在本质上还是在对response建模，因此对uplift的建模还是比较间接，有一定提升的空间。

<img src="../data/img/uplift_one_model_ext.png" style="zoom:50%"/>

### 直接对Uplift进行建模
通过对现有机器学习算法的改造直接对增益效果建模，最流行的是树模型，本文主要讨论树的分裂标准（二叉树）。

<img src="../data/img/uplift_direct_model.png" style="zoom:50%"/>

Uplift Model的建模过程中，虽然我们并没有用户个体的uplift直接的label，但是我们可以通过treatment组和control组转化率的差异来刻画这个uplift，以图中左下角的图为例，我们有T和C两组样本，绿色的样本代表正样本，红色的代表负样本，可以看到在分裂之前T和C两组正负样本的比例比较接近，但是经过一轮特征分裂之后，T和C组内正负样本的比例发生了较大的变化，左子树中T组全是正样本，C组全是负样本，右子树正好相反，C组的正样本居多，意味着左子树的uplift比右子树的uplift更高，即该特征能够很好的把uplift更高和更低的两群人做一个区分。